<a href="https://colab.research.google.com/github/Karthi417/IBM_EDUNET_NM/blob/main/Electricity_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import plotly.graph_objects as go
import calendar

# Load the dataset
data = pd.read_excel('/content/Book1.xlsx')

# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by date
data.sort_values('Date', inplace=True)

# Extract the 'Usage' column for prediction
usage_data = data['Usage'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
usage_data_normalized = scaler.fit_transform(usage_data)

# Create sequences of data for LSTM
sequence_length = 30  # Use the last 30 days' data to predict the next month
X, y = [], []
for i in range(len(usage_data_normalized) - sequence_length):
    X.append(usage_data_normalized[i:i+sequence_length])
    y.append(usage_data_normalized[i+sequence_length])
X, y = np.array(X), np.array(y)

# Reshape X for LSTM input [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train, X_test, y_train, y_test = X[:split_index], X[split_index:], y[:split_index], y[split_index:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values
y_pred_actual = scaler.inverse_transform(y_pred)
y_test_actual = scaler.inverse_transform(y_test)

# Calculate evaluation metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test_actual, y_pred_actual)
mse = mean_squared_error(y_test_actual, y_pred_actual)
r2 = r2_score(y_test_actual, y_pred_actual)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Create traces for actual and predicted usage
actual_trace = go.Scatter(x=data['Date'][split_index+sequence_length:], y=y_test_actual.flatten(), mode='lines', name='Actual Usage')
predicted_trace = go.Scatter(x=data['Date'][split_index+sequence_length:], y=y_pred_actual.flatten(), mode='lines', name='Predicted Usage')

# Convert range objects to lists
train_loss_values = list(range(len(history.history['loss'])))
train_loss = history.history['loss']
val_loss_values = list(range(len(history.history['val_loss'])))
val_loss = history.history['val_loss']

# Create training & validation loss plot
loss_trace = go.Scatter(x=train_loss_values, y=train_loss, mode='lines', name='Train Loss')
val_loss_trace = go.Scatter(x=val_loss_values, y=val_loss, mode='lines', name='Validation Loss')

# Create the loss plot
fig_loss = go.Figure([loss_trace, val_loss_trace])
fig_loss.update_layout(
    title='Model Loss Over Epochs',
    xaxis_title='Epoch',
    yaxis_title='Loss'
)
fig_loss.show()

import plotly.express as px

# Line plot of electricity usage over time
fig_usage_time = px.line(data, x='Date', y='Usage', title='Electricity Usage Over Time')
fig_usage_time.show()

# Box plot of electricity usage by month
fig_usage_month = px.box(data, x='Month', y='Usage', title='Electricity Usage by Month')
fig_usage_month.update_xaxes(title='Month', tickvals=list(range(1, 13)), ticktext=[calendar.month_name[i][:3] for i in range(1, 13)])
fig_usage_month.show()

# Grouped Bar Chart of Electricity Usage by Month
fig_usage_month_grouped = px.bar(data, x='Month', y='Usage', title='Electricity Usage by Month', barmode='group')
fig_usage_month_grouped.update_xaxes(title='Month', tickvals=list(range(1, 13)), ticktext=[calendar.month_name[i][:3] for i in range(1, 13)])
fig_usage_month_grouped.show()

# Create a DataFrame for predicted and actual values
results = pd.DataFrame({
    'Date': data['Date'][split_index+sequence_length:],
    'Actual': y_test_actual.flatten(),
    'Predicted': y_pred_actual.flatten()
})

# Plot predicted vs actual
fig_pred_vs_actual = px.line(results, x='Date', y=['Actual', 'Predicted'], title='Actual vs Predicted Electricity Usage')
fig_pred_vs_actual.show()

import plotly.graph_objects as go
import numpy as np

# Generate some example data for illustration
np.random.seed(0)
n = 100
mae = np.random.uniform(0, 100, n)
mse = np.random.uniform(0, 1000, n)
r_squared = np.random.uniform(0, 1, n)

# Define thresholds for good models
good_mae = 20
good_mse = 200
good_r_squared = 0.9

# Create histogram traces
mae_trace = go.Histogram(x=mae, nbinsx=20, opacity=0.7, name='MAE')
mse_trace = go.Histogram(x=mse, nbinsx=20, opacity=0.7, name='MSE')
r_squared_trace = go.Histogram(x=r_squared, nbinsx=20, opacity=0.7, name='R-squared')

# Create traces for "good" thresholds
good_mae_trace = go.Scatter(x=[good_mae, good_mae], y=[0, 10], mode='lines', name='Good MAE',
                            line=dict(color='green', dash='dash'))
good_mse_trace = go.Scatter(x=[good_mse, good_mse], y=[0, 10], mode='lines', name='Good MSE',
                            line=dict(color='green', dash='dash'))
good_r_squared_trace = go.Scatter(x=[good_r_squared, good_r_squared], y=[0, 10], mode='lines', name='Good R-squared',
                                  line=dict(color='green', dash='dash'))

# Add current positions line
current_mae = mean_absolute_error(y_test_actual, y_pred_actual)
current_mse = mean_squared_error(y_test_actual, y_pred_actual)
current_r2 = r2_score(y_test_actual, y_pred_actual)

current_mae_trace = go.Scatter(x=[current_mae, current_mae], y=[0, 10], mode='lines', name='Current MAE',
                            line=dict(color='red', dash='dash'))
current_mse_trace = go.Scatter(x=[current_mse, current_mse], y=[0, 10], mode='lines', name='Current MSE',
                            line=dict(color='red', dash='dash'))
current_r_squared_trace = go.Scatter(x=[current_r2, current_r2], y=[0, 10], mode='lines', name='Current R-squared',
                                  line=dict(color='red', dash='dash'))

# Create subplots
fig = go.Figure(data=[mae_trace, good_mae_trace, current_mae_trace, mse_trace, good_mse_trace, current_mse_trace,
                       r_squared_trace, good_r_squared_trace, current_r_squared_trace],
                layout=go.Layout(title='Model Evaluation Metrics',
                                 barmode='overlay',
                                 xaxis=dict(title='Metric Value'),
                                 yaxis=dict(title='Frequency')))
fig.show()


Epoch 1/100
10/10 [==============================] - 5s 124ms/step - loss: 0.1367 - val_loss: 0.1092
Epoch 2/100
10/10 [==============================] - 0s 31ms/step - loss: 0.0971 - val_loss: 0.0882
Epoch 3/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0920 - val_loss: 0.0850
Epoch 4/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0922 - val_loss: 0.0834
Epoch 5/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0920 - val_loss: 0.0834
Epoch 6/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0901 - val_loss: 0.0836
Epoch 7/100
10/10 [==============================] - 0s 33ms/step - loss: 0.0890 - val_loss: 0.0825
Epoch 8/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0891 - val_loss: 0.0823
Epoch 9/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0887 - val_loss: 0.0820
Epoch 10/100
10/10 [==============================] - 0s 33ms/step - loss: 0.0882 - val_loss: 0.081